In [1]:
import sys
sys.path.append('../../')
from taxcalc import *
import numpy as np
import pandas as pd
import copy

In [60]:
def make_calculator(refdict = {}, year=2018):
    # Creates a calculator advanced to the given year and calculates tax results
    # Note: Passing an empty dictionary to refdict produces a current law calculator
    assert year in range(2014, 2028)
    assert type(refdict) is dict
    pol = Policy()
    beh = Behavior()
    rec = Records('../../puf.csv')
    if refdict != {}:
        pol.implement_reform(refdict)
    calc1 = Calculator(pol, rec, beh)
    calc1.advance_to_year(year)
    calc1.calc_all()
    return calc1

dropbox_path = 'C:/Users/cody_/Dropbox/TCJA info/notebook & tax calc code/'
# Compile code for distributional analysis
exec(open(dropbox_path + 'distributional_code.py').read())

In [47]:
year_to_use = 2018
param = Calculator.read_json_param_objects('../../taxcalc/reforms/2017_law.json', None)
calc_pre = make_calculator(param['policy'], year_to_use)
calc_tcja = make_calculator({}, year_to_use)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


# Main distribution tables

In [35]:
ranking = {"w_adult": 1, "w_child": 1, "elast_size": 0.5}
scaling = {"w_adult": 1, "w_child": 1, "elast_size": 0}
excluding = ["neginc"]
screening = ["", ""]
tableL = levelTable_km(calc_pre, calc_tcja, ranking, scaling, excluding, screening)
tableC = changeTable_km(calc_pre, calc_tcja, ranking, scaling, excluding, screening)
tableL.to_csv(dropbox_path + 'Main tables/tableL' + str(year_to_use) + '.csv', index=False)
tableC.to_csv(dropbox_path + 'Main tables/tableC' + str(year_to_use) + '.csv', index=False)

In [14]:
tableL

,Income group,"Avg tax rate, pre (%)","Avg tax rate, post (%)","Share of tax liability, pre (%)","Share of tax liability, post (%)","No II tax liability, pre (%)","No II tax liability, post (%)"
0,Bottom decile,-0.899707,-1.167423,-0.013974,-0.019356,98.281102,98.326483
1,Second decile,-7.456847,-8.169143,-0.506268,-0.592048,89.194772,98.495000
2,Third decile,0.759745,-0.644145,0.084631,-0.076595,75.621100,83.170687
3,Fourth decile,7.193430,5.710666,1.160962,0.983837,67.869244,70.446562
4,Fifth decile,11.018166,9.773329,2.449493,2.319338,45.329412,54.994669
5,Sixth decile,13.443744,12.190109,4.027189,3.898019,16.393179,23.406820
6,Seventh decile,16.227628,14.888926,6.448881,6.316076,4.266838,4.782749
7,Eighth decile,19.252646,17.845701,10.414460,10.304683,1.333005,1.214224
8,Ninth decile,22.065753,20.687969,17.024933,17.038807,0.713144,0.628377
9,Next 5%,23.715187,22.377998,13.198458,13.294516,0.668729,0.580716


In [15]:
tableC

,Income group,Change in after-tax income (%),Average tax change ($),Share of the tax change (%),Share receiving tax cut (%),Share receiving tax hike (%)
0,Bottom decile,0.265329,-6.987833,0.065791,7.936882,0.681331
1,Second decile,0.662867,-81.263970,0.765149,42.993686,11.511203
2,Third decile,1.414637,-262.793798,2.474314,59.781396,7.093571
3,Fourth decile,1.597693,-402.153162,3.786289,60.365838,3.111992
4,Fifth decile,1.398979,-465.027563,4.378640,66.997419,4.302238
5,Sixth decile,1.448348,-631.091304,5.941731,80.897558,7.711344
6,Seventh decile,1.598023,-893.903075,8.417307,87.570175,9.712272
7,Eighth decile,1.742404,-1278.932470,12.041581,88.748835,10.373817
8,Ninth decile,1.767880,-1786.378216,16.819294,87.381231,12.127595
9,Next 5%,1.752891,-2501.348250,11.774694,85.629018,13.860365


# Sensitivity analysis tables

### Equivalence scales
 - None
 - Tax unit size
 - Square root of size
 - Oxford scale
 - OECD modified scale

In [39]:
scaling = {"w_adult": 1, "w_child": 1, "elast_size": 0}
excluding = ["neginc"]
screening = ["", ""]
ranking1 = {"w_adult": 1, "w_child": 1, "elast_size": 0}
ranking2 = {"w_adult": 1, "w_child": 1, "elast_size": 1}
ranking3 = {"w_adult": 1, "w_child": 1, "elast_size": 0.5}
ranking4 = {"w_adult": 0.7, "w_child": 0.5, "elast_size": 1}
ranking5 = {"w_adult": 0.5, "w_child": 0.3, "elast_size": 1}

totaltax_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking1, scaling, excluding, screening)
totaltax_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking1, scaling, excluding, screening)
totalinc_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking1, scaling, excluding, screening)
totalinc_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking1, scaling, excluding, screening)

totaltax_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking2, scaling, excluding, screening)
totaltax_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking2, scaling, excluding, screening)
totalinc_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking2, scaling, excluding, screening)
totalinc_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking2, scaling, excluding, screening)

totaltax_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking3, scaling, excluding, screening)
totaltax_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking3, scaling, excluding, screening)
totalinc_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking3, scaling, excluding, screening)
totalinc_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking3, scaling, excluding, screening)

totaltax_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking4, scaling, excluding, screening)
totaltax_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking4, scaling, excluding, screening)
totalinc_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking4, scaling, excluding, screening)
totalinc_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking4, scaling, excluding, screening)

totaltax_pre5 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking5, scaling, excluding, screening)
totaltax_post5 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking5, scaling, excluding, screening)
totalinc_pre5 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking5, scaling, excluding, screening)
totalinc_post5 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking5, scaling, excluding, screening)

rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']

tableS1 = pd.DataFrame({"Income group": rowlabel,
                        "None, pre": totaltax_pre1 / totalinc_pre1 * 100,
                        "None, post": totaltax_post1 / totalinc_post1 * 100,
                        "Tax unit size, pre": totaltax_pre2 / totalinc_pre2 * 100,
                        "Tax unit size, post": totaltax_post2 / totalinc_post2 * 100,
                        "Square root of size, pre": totaltax_pre3 / totalinc_pre3 * 100,
                        "Square root of size, post": totaltax_post3 / totalinc_post3 * 100,
                        "Oxford scale, pre": totaltax_pre4 / totalinc_pre4 * 100,
                        "Oxford scale, post": totaltax_post4 / totalinc_post4 * 100,
                        "OECD modified scale, pre": totaltax_pre5 / totalinc_pre5 * 100,
                        "OECD modified scale, post": totaltax_post5 / totalinc_post5 * 100})
tableS1.to_csv(dropbox_path + 'Sensitivity tables/tableS_ranking.csv', index=False)

### Weighting
 - None
 - Tax unit size
 - Square root of size
 - Number of adults

In [40]:
ranking = {"w_adult": 1, "w_child": 1, "elast_size": 0}
excluding = ["neginc"]
screening = ["", ""]
scaling1 = {"w_adult": 1, "w_child": 1, "elast_size": 0}
scaling2 = {"w_adult": 1, "w_child": 1, "elast_size": 1}
scaling3 = {"w_adult": 1, "w_child": 1, "elast_size": 0.5}
scaling4 = {"w_adult": 1, "w_child": 0, "elast_size": 1}

totaltax_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling1, excluding, screening)
totaltax_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling1, excluding, screening)
totalinc_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling1, excluding, screening)
totalinc_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling1, excluding, screening)

totaltax_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling2, excluding, screening)
totaltax_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling2, excluding, screening)
totalinc_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling2, excluding, screening)
totalinc_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling2, excluding, screening)

totaltax_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling3, excluding, screening)
totaltax_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling3, excluding, screening)
totalinc_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling3, excluding, screening)
totalinc_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling3, excluding, screening)

totaltax_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling4, excluding, screening)
totaltax_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling4, excluding, screening)
totalinc_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling4, excluding, screening)
totalinc_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling4, excluding, screening)

rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']

tableS2 = pd.DataFrame({"Income group": rowlabel,
                        "None, pre": totaltax_pre1 / totalinc_pre1 * 100,
                        "None, post": totaltax_post1 / totalinc_post1 * 100,
                        "Tax unit size, pre": totaltax_pre2 / totalinc_pre2 * 100,
                        "Tax unit size, post": totaltax_post2 / totalinc_post2 * 100,
                        "Square root of size, pre": totaltax_pre3 / totalinc_pre3 * 100,
                        "Square root of size, post": totaltax_post3 / totalinc_post3 * 100,
                        "Number of adults, pre": totaltax_pre4 / totalinc_pre4 * 100,
                        "Number of adults, post": totaltax_post4 / totalinc_post4 * 100})
tableS2.to_csv(dropbox_path + 'Sensitivity tables/tableS_weighting.csv', index=False)

### Income measure
 - Expanded income
 - AGI
 - Pre-transfer income

In [43]:
scaling1 = {"w_adult": 1, "w_child": 1, "elast_size": 0}
ranking = {"w_adult": 1, "w_child": 1, "elast_size": 0}
excluding = ["neginc"]
screening = ["", ""]

totaltax_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling, excluding, screening)
totaltax_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling, excluding, screening)
totalinc_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling, excluding, screening)
totalinc_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling, excluding, screening)

totaltax_pre2 = distTable_km(calc_pre, calc_tcja, 'agi', 'totaltax', 'total1', ranking, scaling, excluding, screening)
totaltax_post2 = distTable_km(calc_pre, calc_tcja, 'agi', 'totaltax', 'total2', ranking, scaling, excluding, screening)
totalinc_pre2 = distTable_km(calc_pre, calc_tcja, 'agi', 'expanded_income', 'total1', ranking, scaling, excluding, screening)
totalinc_post2 = distTable_km(calc_pre, calc_tcja, 'agi', 'expanded_income', 'total2', ranking, scaling, excluding, screening)

totaltax_pre3 = distTable_km(calc_pre, calc_tcja, 'nobenefits', 'totaltax', 'total1', ranking, scaling, excluding, screening)
totaltax_post3 = distTable_km(calc_pre, calc_tcja, 'nobenefits', 'totaltax', 'total2', ranking, scaling, excluding, screening)
totalinc_pre3 = distTable_km(calc_pre, calc_tcja, 'nobenefits', 'expanded_income', 'total1', ranking, scaling, excluding, screening)
totalinc_post3 = distTable_km(calc_pre, calc_tcja, 'nobenefits', 'expanded_income', 'total2', ranking, scaling, excluding, screening)

rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']

tableS3 = pd.DataFrame({"Income group": rowlabel,
                        "Expanded income, pre": totaltax_pre1 / totalinc_pre1 * 100, 
                        "Expanded income, post": totaltax_post1 / totalinc_post1 * 100,
                        "AGI, pre": totaltax_pre2 / totalinc_pre2 * 100, 
                        "AGI, post": totaltax_post2 / totalinc_post2 * 100,
                        "Market income, pre": totaltax_pre3 / totalinc_pre3 * 100, 
                        "Market income, post": totaltax_post3 / totalinc_post3 * 100})
tableS3.to_csv(dropbox_path + 'Sensitivity tables/tableS_income.csv', index=False)

### Excluded groups (all exclude negative income)
 - None
 - Dependents
 - Incomeplete tax units (dependents and married filing separately)
 - Tax > Income

In [45]:
scaling1 = {"w_adult": 1, "w_child": 1, "elast_size": 0}
ranking = {"w_adult": 1, "w_child": 1, "elast_size": 0}
screening = ["", ""]
excluding1 = ["neginc"]
excluding2 = ["neginc", "dependents"]
excluding3 = ["neginc", "dependents", "separate"]
excluding4 = ["neginc", "supertax"]

totaltax_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling, excluding1, screening)
totaltax_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling, excluding1, screening)
totalinc_pre1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling, excluding1, screening)
totalinc_post1 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling, excluding1, screening)

totaltax_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling, excluding2, screening)
totaltax_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling, excluding2, screening)
totalinc_pre2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling, excluding2, screening)
totalinc_post2 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling, excluding2, screening)

totaltax_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling, excluding3, screening)
totaltax_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling, excluding3, screening)
totalinc_pre3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling, excluding3, screening)
totalinc_post3 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling, excluding3, screening)

totaltax_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total1', ranking, scaling, excluding4, screening)
totaltax_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'totaltax', 'total2', ranking, scaling, excluding4, screening)
totalinc_pre4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total1', ranking, scaling, excluding4, screening)
totalinc_post4 = distTable_km(calc_pre, calc_tcja, 'expanded', 'expanded_income', 'total2', ranking, scaling, excluding4, screening)

rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']

tableS4 = pd.DataFrame({"Income group": rowlabel,
                        "None pre": totaltax_pre1 / totalinc_pre1 * 100, 
                        "None, post": totaltax_post1 / totalinc_post1 * 100,
                        "Dependents, pre": totaltax_pre2 / totalinc_pre2 * 100, 
                        "Dependents, post": totaltax_post2 / totalinc_post2 * 100,
                        "Incomplete units, pre": totaltax_pre3 / totalinc_pre3 * 100, 
                        "Incomplete units, post": totaltax_post3 / totalinc_post3 * 100,
                        "Tax > Income, pre": totaltax_pre3 / totalinc_pre3 * 100, 
                        "Tax > Income, post": totaltax_post3 / totalinc_post3 * 100})
tableS4.to_csv(dropbox_path + 'Sensitivity tables/tableS_excluded.csv', index=False)


# Demographic distribution

### Married filing jointly

In [54]:
excluding = ["neginc"]
screening = ["married", ""]
tableD_married = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_married.to_csv(dropbox_path + 'Demographic tables/tableD_married' + str(year_to_use) + '.csv', index=False)

### Not married

In [61]:
excluding = ["neginc"]
screening = ["not_married", ""]
tableD_married = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_married.to_csv(dropbox_path + 'Demographic tables/tableD_notmarried' + str(year_to_use) + '.csv', index=False)

### No children

In [62]:
excluding = ["neginc"]
screening = ["", "0"]
tableD_nokids = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_nokids.to_csv(dropbox_path + 'Demographic tables/tableD_nokids' + str(year_to_use) + '.csv', index=False)

### 1 child

In [64]:
excluding = ["neginc"]
screening = ["", "1"]
tableD_1kid = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_1kid.to_csv(dropbox_path + 'Demographic tables/tableD_1kid' + str(year_to_use) + '.csv', index=False)

## 2 children

In [65]:
excluding = ["neginc"]
screening = ["", "2"]
tableD_2kids = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_2kids.to_csv(dropbox_path + 'Demographic tables/tableD_2kids' + str(year_to_use) + '.csv', index=False)

## 3+ children

In [66]:
excluding = ["neginc"]
screening = ["", "3+"]
tableD_3pluskids = demogTable_km(calc_pre, calc_tcja, excluding, screening)
tableD_3pluskids.to_csv(dropbox_path + 'Demographic tables/tableD_3pluskids' + str(year_to_use) + '.csv', index=False)

# Kakwani index

In [50]:
scaling_options = ['Weight by size', 'Weight by square root', 'Adults']
ranking_options = ['None', 'Size', 'Oxford', 'OECD']
excluding_options = ['Dependents', 'Incomplete units', 'Tax > Income']
income_options = []
rowlabels = ["Main estimates", "Using different equivalence scales", "None", "Size of tax unit", "Oxford", "OECD-modified",
             "Using different weighting systems", "Size of tax unit", "Square root of size", "Number of adults",
             "Using different income measures", "AGI", "Market income",
             "Excluding different groups", "Dependents", "Incomplete units", "Tax > Income"]
kakwani_table = pd.DataFrame({'Category': rowlabels})
year_options = [2018, 2020, 2022, 2024, 2026]
ranking_main = {"w_adult": 1, "w_child": 1, "elast_size": 0.5}
scaling_main = {"w_adult": 1, "w_child": 1, "elast_size": 0}
resultsdict = {}
for y in year_options:
    reslist = []
    calcA = copy.deepcopy(calc_pre)
    calcA.advance_to_year(y)
    calcA.calc_all()
    calcB = copy.deepcopy(calc_tcja)
    calcB.advance_to_year(y)
    calcB.calc_all()
    # Main estimate
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, scaling_main, ["neginc"]))
    reslist.append(0)
    # Alternative equivalence scales
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', {"w_adult": 1, "w_child": 1, "elast_size": 0}, scaling_main, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', {"w_adult": 1, "w_child": 1, "elast_size": 1}, scaling_main, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', {"w_adult": 0.7, "w_child": 0.5, "elast_size": 1}, scaling_main, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', {"w_adult": 0.5, "w_child": 0.3, "elast_size": 1}, scaling_main, ["neginc"]))
    reslist.append(0)
    # Alternative weighting options
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, {"w_adult": 1, "w_child": 1, "elast_size": 1}, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, {"w_adult": 1, "w_child": 1, "elast_size": 0.5}, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, {"w_adult": 1, "w_child": 0, "elast_size": 1}, ["neginc"]))
    reslist.append(0)
    # Alternative income measures
    reslist.append(kakwani(calcA, calcB, 'agi', 'totaltax', ranking_main, scaling_main, ["neginc"]))
    reslist.append(kakwani(calcA, calcB, 'nobenefits', 'totaltax', ranking_main, scaling_main, ["neginc"]))
    reslist.append(0)
    # Alternative excluded groups
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, scaling_main, ["neginc", "dependents"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, scaling_main, ["neginc", "dependents", "separate"]))
    reslist.append(kakwani(calcA, calcB, 'expanded', 'totaltax', ranking_main, scaling_main, ["neginc", "supertax"]))
    kakwani_table[str(y)] = reslist
kakwani_table.to_csv(dropbox_path + 'Sensitivity tables/kakwani.csv', index=False)